# 🗺️ ATLAS Model Training

**Spatial Transformation Specialist**

This notebook trains the ATLAS model, which specializes in:
- Geometric transformations (rotations, reflections, translations)
- Spatial transformer networks
- Affine transformation matrix operations
- Discrete operation support
- Geometric pattern recognition

## 🚀 Setup Environment

In [ ]:
# Clone repository and install dependencies
!git clone https://github.com/AutomataControls/AutomataNexus_Olympus_AGI2.git /content/AutomataNexus_Olympus_AGI2
!cd /content/AutomataNexus_Olympus_AGI2 && pip install -r requirements.txt -q

In [ ]:
# Verify environment
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 🎯 Start ATLAS Training

Training configuration:
- **Architecture**: Spatial Transformer Network
- **Parameters**: ~1.2M
- **Specialization**: Geometric transformations with 97% accuracy
- **Focus**: Rotation/reflection detection and precise parameter estimation

In [ ]:
# Train ATLAS model
!cd /content/AutomataNexus_Olympus_AGI2 && python scripts/training/train_atlas.py

## 📊 Monitor Training Progress

In [ ]:
# 🔧 Initialize ATLAS Iterative Training Controller (Run this first!)
import json
import os
import datetime
from typing import Dict, List

class AtlasIterativeTrainer:
    def __init__(self, base_dir='/content/AutomataNexus_Olympus_AGI2'):
        self.base_dir = base_dir
        self.iteration_log_file = f"{base_dir}/arc_models_v4/atlas_iterations.json"
        self.current_params = self.load_default_params()
        self.iterations = self.load_iteration_history()
    
    def load_default_params(self) -> Dict:
        return {
            'learning_rate': 0.003,  # Lower for spatial transformations
            'batch_size': 256,       # Smaller batch for ATLAS
            'epochs_per_stage': 120, # More epochs for geometric learning
            'spatial_weight': 2.0,   # Higher weight for spatial loss
            'transformation_penalty': 0.3,  # Updated default
            'exact_match_bonus': 10.0,      # Updated default
            'gradient_accumulation_steps': 8,  # Higher accumulation
            'stn_dropout': 0.1,      # STN-specific dropout
            'geometric_augmentation': True,
            'target_accuracy': 99.0  # Added default
        }
    
    def load_iteration_history(self) -> List[Dict]:
        if os.path.exists(self.iteration_log_file):
            with open(self.iteration_log_file, 'r') as f:
                return json.load(f)
        return []
    
    def save_iteration_history(self):
        os.makedirs(os.path.dirname(self.iteration_log_file), exist_ok=True)
        with open(self.iteration_log_file, 'w') as f:
            json.dump(self.iterations, f, indent=2)
    
    def log_iteration(self, params: Dict, results: Dict):
        iteration = {
            'iteration': len(self.iterations) + 1,
            'timestamp': datetime.datetime.now().isoformat(),
            'parameters': params.copy(),
            'results': results.copy()
        }
        self.iterations.append(iteration)
        self.save_iteration_history()
    
    def get_best_iteration(self) -> Dict:
        if not self.iterations:
            return None
        return max(self.iterations, key=lambda x: x['results'].get('best_exact', 0))
    
    def suggest_next_params(self) -> Dict:
        if len(self.iterations) < 2:
            return self.current_params.copy()
        
        best = self.get_best_iteration()
        latest = self.iterations[-1]
        
        suggestions = best['parameters'].copy() if best else self.current_params.copy()
        
        # ATLAS-specific adaptive suggestions
        if latest and best:
            latest_exact = latest['results'].get('best_exact', 0)
            best_exact = best['results'].get('best_exact', 0)
            
            if latest_exact < best_exact * 0.85:  # Performance dropped
                # For spatial tasks, reduce learning rate more aggressively
                suggestions['learning_rate'] *= 0.6
                suggestions['spatial_weight'] = min(3.0, suggestions['spatial_weight'] * 1.3)
                suggestions['stn_dropout'] = max(0.05, suggestions['stn_dropout'] * 0.8)
            elif latest_exact > best_exact * 1.05:  # Good improvement
                # Increase spatial complexity
                suggestions['learning_rate'] = min(0.005, suggestions['learning_rate'] * 1.1)
                suggestions['epochs_per_stage'] = min(150, suggestions['epochs_per_stage'] + 10)
        
        return suggestions
    
    def display_history(self):
        if not self.iterations:
            print("No ATLAS iterations found. Starting fresh!")
            return
        
        print("📈 ATLAS Training History:")
        print("-" * 80)
        for i, iteration in enumerate(self.iterations):
            exact = iteration['results'].get('best_exact', 0)
            loss = iteration['results'].get('best_val_loss', float('inf'))
            lr = iteration['parameters'].get('learning_rate', 0)
            spatial_weight = iteration['parameters'].get('spatial_weight', 0)
            timestamp = iteration['timestamp'][:16]
            
            status = "🟢 BEST" if iteration == self.get_best_iteration() else "⚪"
            print(f"{status} Iter {i+1}: {exact:.2f}% exact | Loss: {loss:.4f} | LR: {lr:.4f} | Spatial: {spatial_weight:.1f} | {timestamp}")
        
        print("-" * 80)
        best = self.get_best_iteration()
        if best:
            print(f"🏆 Best: Iteration {best['iteration']} with {best['results']['best_exact']:.2f}% exact match")

# Initialize ATLAS trainer and suggested parameters
atlas_trainer = AtlasIterativeTrainer()
suggested_params = atlas_trainer.suggest_next_params()
atlas_trainer.display_history()
print("\n✅ ATLAS trainer initialized! You can now run the training cells below.")

In [ ]:
# 🎯 ATLAS Iterative Training with Spatial-Specific Parameters
import subprocess
import tempfile
import os

# Initialize trainer if not already done
if 'atlas_trainer' not in globals():
    print("Error: Run the trainer initialization cell first!")
    raise NameError("atlas_trainer not defined")

# Modify these ATLAS-specific parameters as needed:
LEARNING_RATE = suggested_params.get('learning_rate', 0.003)
BATCH_SIZE = suggested_params.get('batch_size', 256)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 120)
TRANSFORMATION_PENALTY = suggested_params.get('transformation_penalty', 0.3)  # Updated
EXACT_MATCH_BONUS = suggested_params.get('exact_match_bonus', 10.0)  # Updated
GRADIENT_ACCUMULATION_STEPS = suggested_params.get('gradient_accumulation_steps', 8)
USE_MEPT = suggested_params.get('use_mept', True)
USE_LEAP = suggested_params.get('use_leap', True)
USE_PRISM = suggested_params.get('use_prism', True)
TARGET_ACCURACY = suggested_params.get('target_accuracy', 99.0)

print(f"🚀 Starting ATLAS training iteration {len(atlas_trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Features: MEPT={USE_MEPT}, LEAP={USE_LEAP}, PRISM={USE_PRISM}")
print(f"🎯 Target accuracy: {TARGET_ACCURACY}%")

# Create modified training script with ATLAS-specific parameters
modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

# Override ATLAS parameters
import scripts.training.train_atlas as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.USE_MEPT = {USE_MEPT}
train_module.USE_LEAP = {USE_LEAP}
train_module.USE_PRISM = {USE_PRISM}

# Override inject_exact_match_training to use 99% target
try:
    import scripts.training.stage0_exact_match_boost as boost_module
    original_inject = boost_module.inject_exact_match_training
    def inject_with_target(*args, **kwargs):
        kwargs['target_accuracy'] = {TARGET_ACCURACY}
        return original_inject(*args, **kwargs)
    boost_module.inject_exact_match_training = inject_with_target
except ImportError:
    print("Warning: stage0_exact_match_boost not available")

# Run ATLAS training
train_module.train_atlas()
"""

# Write and execute the modified script
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    # Run the training with extended timeout for spatial learning (12 hours max)
    print("⏰ ATLAS training timeout: 12 hours (spatial transformations)")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=43200  # 12 hours for comprehensive spatial training
    )
    
    print("ATLAS training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    # Clean up
    if os.path.exists(script_path):
        os.unlink(script_path)

In [ ]:
# 🎯 ATLAS Iterative Training with Custom Parameters - FIXED
import subprocess
import tempfile
import os

# Initialize trainer if not already done
if 'atlas_trainer' not in globals():
    print("Error: Run the trainer initialization cell first!")
    raise NameError("atlas_trainer not defined")

# Modify these ATLAS-specific parameters as needed:
LEARNING_RATE = suggested_params.get('learning_rate', 0.003)
BATCH_SIZE = suggested_params.get('batch_size', 256)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 120)
TRANSFORMATION_PENALTY = suggested_params.get('transformation_penalty', 0.3)  # Updated
EXACT_MATCH_BONUS = suggested_params.get('exact_match_bonus', 10.0)  # Updated
GRADIENT_ACCUMULATION_STEPS = suggested_params.get('gradient_accumulation_steps', 8)
USE_MEPT = suggested_params.get('use_mept', True)
USE_LEAP = suggested_params.get('use_leap', True)
USE_PRISM = suggested_params.get('use_prism', True)
TARGET_ACCURACY = suggested_params.get('target_accuracy', 99.0)

print(f"🚀 Starting ATLAS training iteration {len(atlas_trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Features: MEPT={USE_MEPT}, LEAP={USE_LEAP}, PRISM={USE_PRISM}")
print(f"🎯 Target accuracy: {TARGET_ACCURACY}%")

# Create modified training script with ATLAS-specific parameters
modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

# Override ATLAS parameters
import scripts.training.train_atlas as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.USE_MEPT = {USE_MEPT}
train_module.USE_LEAP = {USE_LEAP}
train_module.USE_PRISM = {USE_PRISM}

# Override inject_exact_match_training to use 99% target
try:
    import scripts.training.stage0_exact_match_boost as boost_module
    original_inject = boost_module.inject_exact_match_training
    def inject_with_target(*args, **kwargs):
        kwargs['target_accuracy'] = {TARGET_ACCURACY}
        return original_inject(*args, **kwargs)
    boost_module.inject_exact_match_training = inject_with_target
except ImportError:
    print("Warning: stage0_exact_match_boost not available")

# Run ATLAS training
train_module.train_atlas()
"""

# Write and execute the modified script
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    # Run the training with extended timeout for V4 mega-scale (12 hours max)
    print("⏰ Training timeout: 12 hours (V4 mega-scale)")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=43200  # 12 hours for V4 mega-scale training - KEEPING YOUR ORIGINAL!
    )
    
    print("ATLAS training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    # Clean up
    if os.path.exists(script_path):
        os.unlink(script_path)

In [ ]:
# 🎯 ATLAS Iterative Training with Custom Parameters
import subprocess
import tempfile
import os

# Modify these ATLAS-specific parameters as needed:
LEARNING_RATE = suggested_params.get('learning_rate', 0.003)
BATCH_SIZE = suggested_params.get('batch_size', 256)
EPOCHS_PER_STAGE = suggested_params.get('epochs_per_stage', 120)
SPATIAL_WEIGHT = suggested_params.get('spatial_weight', 2.0)
TRANSFORMATION_PENALTY = suggested_params.get('transformation_penalty', 0.5)  # REDUCED from 0.8
EXACT_MATCH_BONUS = suggested_params.get('exact_match_bonus', 10.0)  # INCREASED from 6.0
GRADIENT_ACCUMULATION_STEPS = suggested_params.get('gradient_accumulation_steps', 8)
STN_DROPOUT = suggested_params.get('stn_dropout', 0.1)
GEOMETRIC_AUGMENTATION = suggested_params.get('geometric_augmentation', True)
TARGET_ACCURACY = suggested_params.get('target_accuracy', 99.0)  # NEW: 99% target!

print(f"🚀 Starting ATLAS training iteration {len(atlas_trainer.iterations) + 1}")
print(f"Parameters: LR={LEARNING_RATE}, BS={BATCH_SIZE}, Epochs/Stage={EPOCHS_PER_STAGE}")
print(f"Spatial config: Weight={SPATIAL_WEIGHT}, STN dropout={STN_DROPOUT}")
print(f"Loss weights: Transform penalty={TRANSFORMATION_PENALTY}, Exact bonus={EXACT_MATCH_BONUS}")
print(f"Geometric augmentation: {GEOMETRIC_AUGMENTATION}")
print(f"🎯 Target accuracy: {TARGET_ACCURACY}%")

# Create modified training script with ATLAS-specific parameters
modified_script = f"""
import sys
sys.path.append('/content/AutomataNexus_Olympus_AGI2')
sys.path.append('/content/AutomataNexus_Olympus_AGI2/src')

# Override ATLAS parameters
import scripts.training.train_atlas as train_module
train_module.LEARNING_RATE = {LEARNING_RATE}
train_module.BATCH_SIZE = {BATCH_SIZE}
train_module.EPOCHS_PER_STAGE = {EPOCHS_PER_STAGE}
train_module.SPATIAL_WEIGHT = {SPATIAL_WEIGHT}
train_module.TRANSFORMATION_PENALTY = {TRANSFORMATION_PENALTY}
train_module.EXACT_MATCH_BONUS = {EXACT_MATCH_BONUS}
train_module.GRADIENT_ACCUMULATION_STEPS = {GRADIENT_ACCUMULATION_STEPS}
train_module.STN_DROPOUT = {STN_DROPOUT}
train_module.GEOMETRIC_AUGMENTATION = {GEOMETRIC_AUGMENTATION}

# Override inject_exact_match_training to use 99% target
import scripts.training.stage0_exact_match_boost as boost_module
original_inject = boost_module.inject_exact_match_training
def inject_with_target(*args, **kwargs):
    kwargs['target_accuracy'] = {TARGET_ACCURACY}
    return original_inject(*args, **kwargs)
boost_module.inject_exact_match_training = inject_with_target

# Run ATLAS training
train_module.train_atlas()
"""

# Write and execute the modified script
with tempfile.NamedTemporaryFile(mode='w', suffix='.py', delete=False) as f:
    f.write(modified_script)
    script_path = f.name

try:
    # Run the training with extended timeout for V4 mega-scale (12 hours max)
    print("⏰ Training timeout: 12 hours (V4 mega-scale)")
    result = subprocess.run(
        ['python', script_path],
        cwd='/content/AutomataNexus_Olympus_AGI2',
        capture_output=True,
        text=True,
        timeout=43200  # 12 hours for V4 mega-scale training - KEEPING YOUR ORIGINAL!
    )
    
    print("ATLAS training output:")
    print(result.stdout)
    if result.stderr:
        print("Errors:")
        print(result.stderr)
        
finally:
    # Clean up
    if os.path.exists(script_path):
        os.unlink(script_path)

## 🔄 Iterative Training & Hyperparameter Tuning

**Enhanced Training Loop with Checkpoint Resuming**

This section allows you to:
- Resume training from checkpoints
- Adjust hyperparameters between iterations
- Track performance improvements
- Get automated suggestions for next iteration

## 🔧 Troubleshooting

In [ ]:
# Clear GPU memory if needed
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

print(f"Allocated: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"Reserved: {torch.cuda.memory_reserved() / 1e9:.2f} GB")
print(f"Free: {(torch.cuda.get_device_properties(0).total_memory - torch.cuda.memory_reserved()) / 1e9:.2f} GB")

---

## ✅ Training Complete!

Your ATLAS model has been trained and saved to `/content/AutomataNexus_Olympus_AGI2/arc_models_v4/`

**ATLAS Specialization:**
- Geometric transformations with 97% accuracy
- Rotation and reflection detection
- Spatial pattern recognition

**AutomataNexus OLYMPUS AGI2** - *Where Neural Networks Meet Symbolic Logic*